Reference
* https://developers.google.com/optimization/flow/assignment_min_cost_flow

In [1]:
# Install the latest version of ortools
# python -m pip install --upgrade --user ortools
import ortools
print(ortools.__version__)

9.9.3963


In [2]:
import numpy as np
from ortools.graph.python import min_cost_flow

In [3]:
# Instantiate a SimpleMinCostFlow solver.
smcf = min_cost_flow.SimpleMinCostFlow()

In [4]:
# Define the directed graph for the flow
start_nodes = (
    [0, 0] # 0 can go to 1 or 2
    + [1, 1, 1, 1, 1, 2, 2, 2, 2, 2] # 1 or 2 can go to 3, 4, 5, 6, 7
    + [3, 3, 3, 3, 3, 4, 5, 6, 7]
    + [4, 5, 6, 7] # If 3 to X and then to 8
)
end_nodes = (
    [1, 2] # 0 can go to 1 or 2
    + [3, 4, 5, 6, 7, 3, 4, 5, 6, 7] # 1 or 2 can go to 3, 4, 5, 6, 7
    + [4, 5, 6, 7, 8, 8, 8, 8, 8]
    + [8, 8, 8, 8] # If 3 to X (4, 5, 6, or 7) and then to 8
)
# Capacities and costs on each route
capacities = (
    [1, 1]
    + [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    + [1, 1, 1, 1, 1, 1, 1, 1, 1]
    + [1, 1, 1, 1]
)
costs = ( 
    [1, 0] # Direct (2) actually skips node so no cost
    + [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
    + [1, 1, 1, 1, 1, 1, 1, 1, 1]
    + [1, 1, 1, 1]
)
source = 0
sink = 8
tasks = 2
supplies = [tasks, 0, 0, 0, 0, 0, 0, 0, -tasks]

In [5]:
# Add each arc.
for i in range(len(start_nodes)):
    smcf.add_arc_with_capacity_and_unit_cost(
        start_nodes[i], end_nodes[i], capacities[i], costs[i]
    )
# Add node supplies.
for i in range(len(supplies)):
    smcf.set_node_supply(i, supplies[i])

In [6]:
# Find the min cost flow.
status = smcf.solve()

In [7]:
# %%time
if status == smcf.OPTIMAL:
    print("Total cost = ", smcf.optimal_cost())
    print()
    for arc in range(smcf.num_arcs()):
        if smcf.flow(arc) > 0:
            print(
                "Node %d flows to node %d.  Cost = %d"
                % (smcf.tail(arc), smcf.head(arc), smcf.unit_cost(arc))
            )
else:
    print("There was an issue with the min cost flow input.")
    print(f"Status: {status}")

Total cost =  5

Node 0 flows to node 1.  Cost = 1
Node 0 flows to node 2.  Cost = 0
Node 1 flows to node 6.  Cost = 1
Node 2 flows to node 7.  Cost = 1
Node 6 flows to node 8.  Cost = 1
Node 7 flows to node 8.  Cost = 1
